In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import plotly.express as px
import matplotlib.pyplot as plt
from collections import namedtuple
import matplotlib.patches as patches
import networkx as nx

In [ ]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

In [ ]:
df = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/oligosdegradsample2-1.xlsx')
df.shape
df = thermo_df(df)
plot_zone(df)

In [ ]:
df_sample = df[(df.Mass < 22000)]
plot_zone(df_sample)

In [ ]:
idxs = list()
# create our callback function
def on_selection(trace, points, selector):
    print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

def on_click(trace, points, selector):
#     print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

f = zone_selection(df_sample, on_selection=on_selection, on_click=on_click)
f

In [ ]:
df_chosen = df_sample.iloc[idxs]
df_chosen.shape

In [ ]:
# df_hi = df_chosen
df_lo = df_chosen

In [ ]:
x = df_hi.Mass
y = df_hi.RT
z = np.polyfit(x, y, 4)
def f_hi2(x):
    return z[0] * x**4 + z[1]*x**3 + z[2]*x**2 + z[3]*x + z[4] + 0.1
y = f_hi2(x)
plt.figure(figsize=(16, 12))
plt.scatter(df_sample.Mass, df_sample.RT)
plt.plot(x, y, "r-")

plt.scatter(x, y)
plt.show()

In [ ]:
x = df_lo.Mass
y = df_lo.RT
z_lo = np.polyfit(x, y, 4)
def f_lo2(x):
    return z_lo[0] * x**4 + z_lo[1]*x**3 + z_lo[2]*x**2 + z_lo[3]*x + z_lo[4] - 0.1
#     return z_lo[0] * x**3 + z_lo[1]*x**2 + z_lo[2]*x + z_lo[3]
y = f_lo2(x)
plt.figure(figsize=(16, 12))
plt.plot(x, y, "r-")
plt.scatter(df_sample.Mass, df_sample.RT)
plt.scatter(x, y)
plt.show()

In [ ]:
_, ax = plt.subplots(figsize=(16, 12))
ax.scatter(df_sample.Mass, df_sample.RT)

ax.scatter(df_hi.Mass, df_hi.RT, c='blue')
ax.plot(df_hi.Mass, f_hi2(df_hi.Mass), c='blue')

ax.scatter(df_lo.Mass, df_lo.RT, c='green')
ax.plot(df_lo.Mass, f_lo2(df_lo.Mass), c='green')

In [ ]:
sample_rt_min = df_sample.RT.min()
sample_rt_max = df_sample.RT.max()

In [ ]:
filter_min = 0.1
filter_nums = 10

def bin_range2(df, left, right, f_hi, f_lo, bin_num):
    rt = df.RT
    if rt.empty:
        return 0.0, 0.0
#     rt_min = rt.min()
#     rt_max = rt.max()
    rt_min = f_lo(left)
    rt_min = sample_rt_min if rt_min < sample_rt_min else rt_min
    rt_max = f_hi(right)
    rt_max = sample_rt_max if rt_max > sample_rt_max else rt_max
    rt_height = rt_max - rt_min
#     print(rt_min, rt_max, rt_height)
    
    bin_size = rt_height / filter_nums
    bin_size = filter_min if bin_size < filter_min else bin_size
    
    rt_hi = rt_max - bin_size * (bin_num - 1)
    rt_lo = rt_hi - bin_size
#     print(type(rt_lo), type(bin_size))
    if np.less(rt_lo, rt_min):
        print('Reach the last bin.')
        rt_lo = rt_min
        rt_hi = rt_lo + bin_size
    print('df_win count {} winRT {}-{} filterRT lo-hi {}-{}'.format(df.shape[0], rt_min, rt_max, rt_lo, rt_hi))
    return rt_lo, rt_hi

In [ ]:
def create_rectangle(left, right, lo, hi):
    rect = patches.Rectangle((left, lo), right-left, hi-lo)
    return rect

In [ ]:
cursor = 0
mass_win_size = 10
bin_num = 1
last_rt_lo = 0.0

# plt.figure(figsize=(16, 9))
fig, ax = plt.subplots(1, figsize=(16, 9))
ax.scatter(df.Mass, df.RT)
filters = list()
while cursor < df_sample.Mass.max():
    # do sth
#     print('processing win {}-{}'.format(cursor, cursor+mass_win_size))
    df_win = df[(df.Mass >= cursor) & (df.Mass < cursor + mass_win_size)]
    rt_lo, rt_hi = bin_range2(df_win, cursor, cursor+mass_win_size, f_hi2, f_lo2, bin_num)
    if rt_lo < last_rt_lo:
        rt_lo = last_rt_lo
        if rt_lo >= rt_hi:
            cursor += mass_win_size
            continue
            
    if np.isclose(rt_lo, 0.0) or np.isclose(rt_hi, 0.0):
        cursor += mass_win_size
        continue
    
    rect = create_rectangle(cursor, cursor+mass_win_size, rt_lo, rt_hi)
    ax.add_patch(rect)
    
    last_rt_lo = rt_lo
    df_filter = df_win[(df_win.RT>=rt_lo) & (df_win.RT<=rt_hi)]
#     print(cursor, df_filter.shape)
    filters.append(df_filter)
    cursor += mass_win_size

plt.show()

In [ ]:
df_filters = pd.concat(filters)
plt = plot_zones(df_sample, df_filters)
plt.plot(df_hi.Mass, f_hi2(df_hi.Mass))
plt.plot(df_lo.Mass, f_lo2(df_lo.Mass))
# plt.savefig('/Users/bryan/Documents/Presentations/Jan 12 2021/filter_{}_bin_{}_filters_{}.png'.format(filter_min, bin_num, filter_nums), transparent=True)

In [ ]:
bcr_5p = base_calling_random(df_filters)
plt, _ = plot_basecalling(*bcr_5p, annotate=False)
# plt.savefig('/Users/bryan/Documents/Presentations/Jan 12 2021/filter_bc_{}_bin_{}_filters_{}.png'.format(filter_min, bin_num, filter_nums), transparent=True)

In [ ]:
G = nx.DiGraph()

df_bcr = bcr_5p[0]
mass_pairs = bcr_5p[1]
for t in mass_pairs:
    G.add_edge(t[1], t[0], base=t[-1])

path = longest_path(G)
seqs = [G.edges[path[i], path[i+1]]['base'] for i in range(len(path)-1)]
print('longest sequence: ' + ''.join(seqs))
print(path)

In [ ]:
def longest_path(G):
    dist = {} # stores [node, distance] pair
    for node in nx.topological_sort(G):
        # pairs of dist,node for all incoming edges
        pairs = [(dist[v][0]+1,v) for v in G.pred[node]] 
        if pairs:
            dist[node] = max(pairs)
        else:
            dist[node] = (0, node)
    node,(length,_)  = max(dist.items(), key=lambda x:x[1])
    path = []
    while length > 0:
        path.append(node)
        length,node = dist[node]
    return list(reversed(path))

# Mass Sum -- Phe

In [ ]:
# df_phe = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/Data/Excels/total_tRNA_acid_degradated_20to40_component_only.xlsx')
df_phe = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/Data/Excels/Sigma_tRNA_standard_150pmol_Degradation.xlsx')
# df_phe = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/Data/Excels/Sigma_tRNA_standard_150pmol_Degradation_all_level.xlsx')
# df_phe = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/Data/Excels/tRNA_Phe_0724.xlsx')
# df_phe = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/Data/Excels/tRNA_Phe_0724_COMBINE2.xlsx')
print(df_phe.shape)
df_phe = thermo_df(df_phe)
plot_zone(df_phe)

In [ ]:
df_phe_25k = df_phe[(df_phe.Mass > 23000) & (df_phe.Mass < 26000)]
px.scatter(x=df_phe_25k.Mass, y=df_phe_25k.Vol)

In [ ]:
df_phe[df_phe.Mass > 24000].sort_values('Mass')

In [ ]:
df_sample = df_phe[df_phe.Mass < 25000]

In [ ]:
full_mass = 24252.315242 #24581.374101 
df_3p_cc, df_5p_cc = computational_data_seperation(df_sample, df_sample, full_mass, ignore_endpoints=True)

In [ ]:
full_mass =24252.315242 - 14 #24581.374101 
df_3p_cc_m1m, df_5p_cc_m1m = computational_data_seperation(df_sample, df_sample, full_mass, ignore_endpoints=True)

In [ ]:
full_mass = 24252.315242 - 14*2 #24238.334749 #24252.315242 #24581.374101 
df_3p_cc_m2m, df_5p_cc_m2m = computational_data_seperation(df_sample, df_sample, full_mass, ignore_endpoints=True)

In [ ]:
full_mass = 24252.315242 + 14*2 #24238.334749 #24252.315242 #24581.374101 
df_3p_cc_a2m, df_5p_cc_a2m = computational_data_seperation(df_sample, df_sample, full_mass, ignore_endpoints=True)

In [ ]:
full_mass = 24581.374101 
df_3p_cca, df_5p_cca = computational_data_seperation(df_sample, df_sample, full_mass, ignore_endpoints=True)

In [ ]:
full_mass = 24581.374101 - 14 #24238.334749 #24252.315242 #24581.374101 
df_3p_cca_m1m, df_5p_cca_m1m = computational_data_seperation(df_sample, df_sample, full_mass, ignore_endpoints=True)

In [ ]:
full_mass = 24581.374101 + 14*2 #24238.334749 #24252.315242 #24581.374101 
df_3p_cca_a2m, df_5p_cca_a2m = computational_data_seperation(df_sample, df_sample, full_mass, ignore_endpoints=True)

In [ ]:
plt.figure(figsize=(24, 18))
plt.scatter(df_sample.Mass, df_sample.RT)
plt.scatter(df_3p_cc.Mass, df_3p_cc.RT)
# plot_zone(df_3p_cc, trend=True)
plt.scatter(df_3p_cca.Mass, df_3p_cca.RT)
# px.scatter(df_3p_cc, x='Mass', y='RT')
# px.scatter(df_3p_cca, x='Mass', y='RT')
# plt.scatter(df_3p_cc_m1m.Mass, df_3p_cc_m1m.RT)
# plt.scatter(df_3p_cc_m1m.Mass, df_3p_cc_m1m.RT)

In [ ]:
bcr_5p = base_calling_random(df_3p_cca)
plt, _ = plot_basecalling2(*bcr_5p, annotate=False)

In [ ]:
df_3p_cc.loc[:,'type'] = 'Reg'
df_3p_cca.loc[:,'type'] = 'Methyl'
df_merge = pd.concat([df_3p_cc, df_3p_cca])
fig = px.scatter(df_merge, x='Mass', y='RT', color='type')
fig.update_layout(
    width=960*1.2,
    height=720*1.2,
    margin=dict(l=0, r=0, t=20, b=0),
    paper_bgcolor="LightSteelBlue",
)

In [ ]:
df_total = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/total_tRNA_acid_deg_oligosdegcoplex4.xlsx')
df_total.shape
df_total = thermo_df(df_total)
plot_zone(df_total)

In [ ]:
df_total_25k = df_total[(df_total.Mass > 22000) & (df_total.Mass < 26000)]

In [ ]:
px.scatter(x=df_total_25k.Mass, y=df_total_25k.Vol)

In [ ]:
df_sample = df_total[df_total.Mass < 22000]
df_sample.shape
plot_zone(df_sample)

In [ ]:
full_mass = 23904.22164 #24252.315242 #24581.374101 
df_3p_cc, df_5p_cc = computational_data_seperation(df_sample, df_sample, full_mass, ignore_endpoints=True)

In [ ]:
plt.figure(figsize=(16, 12))
plt.scatter(df_sample.Mass, df_sample.RT)
plt.scatter(df_3p_cc.Mass, df_3p_cc.RT)

In [ ]:

px.scatter(x=df_3p_cc.Mass, y=df_3p_cc.RT)

In [ ]:
full_mass = 24581.374101 
df_3p_cca, df_5p_cca = computational_data_seperation(df_sample, df_sample, full_mass, ignore_endpoints=True)

# Utils

In [ ]:
def plot_basecalling2(df, mass_pairs, endpoints=pd.DataFrame(), annotate=True, plt=None):
#     if in_plt:
#         plt = in_plt
#     else:
    if not plt:
        plt = matplotlib.pyplot
    fig = plt.figure(figsize=(24, 18))
#     plt.xlabel('Mass (Da)')
#     plt.ylabel('Retention Time (min)')
    plt.xlabel('Monoisotopic Mass (Da)', fontname="Arial", fontsize=15, color='black')
    plt.ylabel('Retention Time (min)', fontname="Arial", fontsize=15, color='black')
#     plt.xticks(range(0, 25001, 2500), fontname="Arial", size=13, color='black')
#     plt.yticks(range(0, 25, 2), fontname="Arial", size=13, color='black')
    plt.xticks(fontname="Arial", size=13, color='black')
    plt.yticks(fontname="Arial", size=13, color='black')
        
    plt.scatter(df.Mass, df.RT)

    for t in mass_pairs:
        df_pair = df[df.Mass.isin(t)]
        if df_pair.empty:
            continue
        plt.plot(df_pair.Mass, df_pair.RT, 'green')
        
        idmax = df_pair['Mass'].idxmax()
        plt.annotate(s=t[2], size=15, xy=(df_pair.loc[idmax].Mass, df_pair.loc[idmax].RT), 
                     textcoords="offset points", xytext=(-10, 10), ha='center', color='C0')
        
        if not annotate:
            continue
            
        mass = '{:.2f}'.format(df_pair.loc[idmax].Mass)
        plt.annotate(s=mass, size=13, xy=(df_pair.loc[idmax].Mass, df_pair.loc[idmax].RT), 
                     textcoords="offset points", xytext=(10, -20), ha='center')

    if not endpoints.empty:
        plt.scatter(endpoints.Mass, endpoints.RT, color='r')
        print(endpoints[['Mass', 'RT', 'Vol']])
#     plt.show()
    return plt, fig